In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [13]:
data = pd.read_json('../CrawlData/bonbanh.json')

In [12]:
data.iloc[0]

url                  https://bonbanh.com/xe-honda-jazz-v-2018-4034703
title                            Xe\tHonda Jazz\tV\t2018\t- 429 Triệu
description         [Honda Jazz 1.5AT, Số tự động , Nhập Thái Lan,...
seller                                                    Hiền Toyota
date                                Đăng ngày 6/12/2021\t. Xem 4 lượt
seller_address      Địa chỉ: C3/21H Phạm Hùng, Bình Hưng, Bình Chá...
url_image           https://bonbanh.com/uploads/users/128387/car/4...
origin                                                      Nhập khẩu
status                                                     Xe đã dùng
car_model                                                   Hatchback
km_number                                                   20,000 Km
exterior_color                                                  Trắng
interior_color                                                    Đen
number_door                                                     5 cửa
number_seat         

In [14]:
for x in tqdm(range(data.shape[0])):
    data.loc[x,'description'] = ' '.join(data.loc[x,'description'])

100%|██████████████████████████████████████████████████████| 40563/40563 [00:05<00:00, 6955.82it/s]


In [15]:
for x in tqdm(range(data.shape[0])):
    tmp = data.km_number.iloc[x].split()
    data.loc[x,'car_mileage'] = int(tmp[0].replace(',',''))

100%|██████████████████████████████████████████████████████| 40563/40563 [00:11<00:00, 3516.22it/s]


In [16]:
for x in tqdm(range(data.shape[0])):
    tmp = data.number_seat.iloc[x].split()
    data.loc[x,'car_seats'] = int(tmp[0].replace(',',''))

100%|██████████████████████████████████████████████████████| 40563/40563 [00:12<00:00, 3372.87it/s]


In [17]:
for x in tqdm(range(data.shape[0])):
    tmp = data.title.iloc[x].replace('-','').strip()
    tmp = tmp.split('\t')
    if(len(tmp) == 5):
        cname = tmp[1].strip()
        ctype = tmp[2].strip()
        car_year = tmp[3].strip()
        priceStr = tmp[4].strip().split()
    elif (len(tmp) == 4):
        cname = tmp[1].strip()
        ctype = ''
        car_year = tmp[2].strip()
        priceStr = tmp[3].strip().split()
        
    car_year = [int(s) for s in car_year.split() if s.isdigit()][0]
    data.loc[x,'cname'] = cname
    data.loc[x,'ctype'] = ctype
    data.loc[x,'car_year'] = car_year
    price = 0
    for i in range(0,len(priceStr),2):
        x1 = int(priceStr[i])
        x2 = priceStr[i+1]
        if x2 == 'Tỷ':
            price +=x1*1e9
        elif x2 =='Triệu':
            price += x1 *1e6
    data.loc[x,'car_price'] = price

100%|███████████████████████████████████████████████████████| 40563/40563 [05:33<00:00, 121.49it/s]


In [18]:
data['car_type'] = data['car_model']

In [19]:
listCity = ['Gia Lai', 'TP HCM', 'Thanh Hóa', 'Hà Nội', 'Đồng Nai',
       'Bình Dương', 'Lâm Đồng', 'An Giang', 'Hải Phòng', 'Cần Thơ',
       'Bắc Ninh', 'Tây Ninh', 'Bắc Giang', 'Tiền Giang', 'Đăk Lăk',
       'Đồng Tháp', 'Sóc Trăng', 'Bà Rịa Vũng Tàu', 'Đà Nẵng',
       'Vĩnh Phúc', 'Nghệ An', 'Phú Thọ', 'Hải Dương', 'Yên Bái',
       'Điện Biên', 'Nam Định', 'Khánh Hòa', 'Bình Thuận', 'Ninh Bình',
       'Bắc Kạn', 'Vĩnh Long', 'Sơn La', 'Thái Bình', 'Quảng Ninh',
       'Thái Nguyên', 'Hưng Yên', 'Cao Bằng', 'Quảng Trị', 'Bình Định',
       'Kon Tum', 'Hà Nam', 'Hà Tĩnh', 'Thừa Thiên Huế', 'Quảng Ngãi',
       'Bình Phước', 'Lạng Sơn', 'Quảng Bình', 'Đăk Nông', 'Phú Yên',
       'Hậu Giang', 'Long An', 'Quảng Nam', 'Bến Tre', 'Kiên Giang',
       'Lào Cai', 'Hòa Bình', 'Tuyên Quang', 'Bạc Liêu', 'Hà Giang',
       'Trà Vinh', 'Cà Mau', 'Ninh Thuận', 'Lai Châu']

In [20]:
for x in listCity:
    id = data[data.seller_address.str.contains(x)].index
    data.loc[id,'seller_location'] = x

In [21]:
for x in tqdm(range(data.shape[0])):
    tmp = data.date.iloc[x].replace('Đăng ngày\t','')
    tmp = tmp.replace('Đăng ngày ','')
    tmp = tmp.split('\t')
    data.loc[x,'createdAt'] = tmp[0]

100%|███████████████████████████████████████████████████████| 40563/40563 [01:20<00:00, 504.45it/s]


In [22]:
data['car_fuel'] = data['engine']

In [23]:
data['car_origin'] = data['origin']

In [24]:
data['car_condition'] = data['status']

In [25]:
data['car_color'] = data['exterior_color']

In [26]:
data['car_gearbox'] = data['gear']

In [27]:
cols = ['description', 'car_origin', 'car_condition', 'car_mileage', 'car_color',
       'car_seats', 'car_gearbox', 'car_year', 'car_price', 'car_type', 'createdAt',
       'cname', 'car_fuel', 'seller_location','url_image']

In [28]:
tmp = data[cols]

In [29]:
tmp.iloc[0]

description        Honda Jazz 1.5AT Số tự động  Nhập Thái Lan Odo...
car_origin                                                 Nhập khẩu
car_condition                                             Xe đã dùng
car_mileage                                                  20000.0
car_color                                                      Trắng
car_seats                                                        5.0
car_gearbox                                               Số tự động
car_year                                                      2018.0
car_price                                                429000000.0
car_type                                                   Hatchback
createdAt                                                  6/12/2021
cname                                                     Honda Jazz
car_fuel                                                 Xăng\t1.5 L
seller_location                                               TP HCM
url_image          https://bonbanh

In [30]:
tmp.to_csv('../Data/bonbanh_ps.csv',index=False)